In [21]:
# !pip install -U torchdata
# !pip install -U spacy
# !python -m spacy download en_core_web_lg
# !python -m spacy download de_core_news_lg
# !pip install portalocker
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

In [22]:
#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU: Tesla P100-PCIE-16GB


In [23]:
PATH="/kaggle/input/webis-clickbait-22/"
OUTPATH="/kaggle/working/"
# PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=8
T5_model = "/kaggle/input/t5-dataset-proj/T5/checkpoint-3200"
sep='[SEP]'

In [24]:
# data=pd.read_json(PATH+"train.jsonl",lines=True,encoding='utf-8')
# data.head()

In [25]:
def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')


    y = data["spoiler"]
    Y=[]
    for i in y:
        Y.append(" ".join(i))
        
    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    for i in dic:
        # print(i)
        i['postText']=i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']=targetparah
        # if(targetDescription is None or len(targetDescription)==0):
        #     targetDescription='none'
        #     desccount+=1
        # i['targetDescription']='targetDescription: '+targetDescription
        # targetkey=i['targetKeywords']
        # if(targetkey is None or len(targetkey)==0):
        #     targetkey='none'
        #     keycount+=1
        # i['targetKeywords']='targetKeywords: '+targetkey
        i['question']="question : "+i['postText']+"?"
        i['context']="context : "+i['targetParagraphs']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    # print("desccount: ",desccount)
    # print("keycount: ",keycount)
    X_q = [i['question'] for i in dic]
    X_q=np.array(X_q)
    X_c = [i['context'] for i in dic]
    X_c=np.array(X_c)

    return X_q[:100],X_c[:100],Y[:100]
#     return X_q,X_c,Y

In [26]:
x_train_q,x_train_c,y_train = get_split_data(PATH+'train.jsonl')
x_test_q,x_test_c,y_test = get_split_data(PATH+'test.jsonl',False)
x_val_q,x_val_c,y_val = get_split_data(PATH+'validation.jsonl',False)

In [27]:
x_train_q.shape,x_train_c.shape,len(y_train),x_test_q.shape,x_test_c.shape,len(y_test),x_val_q.shape,x_val_c.shape,len(y_val)

((100,), (100,), 100, (100,), (100,), 100, (100,), (100,), 100)

In [28]:
# create pd dataframe
train_df = pd.DataFrame(list(zip(x_train_q, x_train_c, y_train)), columns =['question', 'context', 'spoiler'])
val_df = pd.DataFrame(list(zip(x_val_q, x_val_c, y_val)), columns =['question', 'context', 'spoiler'])
test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test)), columns =['question', 'context', 'spoiler'])


In [29]:
train_df["question"][0],train_df["context"][0],train_df["spoiler"][0]

('question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?',
 'context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we

In [30]:
# t5
tokenizer = AutoTokenizer.from_pretrained(T5_model)
model = AutoModelWithLMHead.from_pretrained(T5_model)


/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [31]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
train_data = Dataset(pa.Table.from_pandas(train_df))
val_data = Dataset(pa.Table.from_pandas(val_df))
test_data = Dataset(pa.Table.from_pandas(test_df))
train_data,val_data,test_data

(Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 100
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 100
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 100
 }))

In [32]:

train_data[0]

{'question': 'question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?',
 'context': 'context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get 

In [33]:
def preproc(data):
    q=[]
    c=[]
    a=[]
    # print(len(data))
    # print(len(data['question']))
    # print(len(data['context']))
    # print(len(data['spoiler']))
    for i in range(len(data['question'])):
        q.append(data['question'][i])
        c.append(data['context'][i])
        a.append(data['spoiler'][i])

    model_inputs=tokenizer(q,c,text_target=a,return_tensors='pt',padding=True,truncation=True,max_length=512)
    return model_inputs


In [34]:
# train_data=Dataset.from_pandas(train_df)
tokenized_train=train_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_val=val_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_test=test_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_train,tokenized_val,tokenized_test

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

(Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 100
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 100
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 100
 }))

In [35]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=T5_model)

In [36]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [37]:
model = AutoModelForSeq2SeqLM.from_pretrained(T5_model)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [38]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [39]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred, ref = postprocess_text(decoded_preds, decoded_labels)

    # cal bleu, meteor and bert
    bleu_score = bleu.compute(predictions=pred, references=ref)
#     bleu1 = corpus_bleu(ref,pred,weights=(1,0,0,0))
#     bleu2 = corpus_bleu(ref,pred,weights=(0.5,0.5,0))
#     bleu3 = corpus_bleu(ref,pred,weights=(0.33,0.33,0.33,0)
#     bleu4 = corpus_bleu(ref,pred,weights=(0.25,0.25,0.25,0.25))

    print(bleu_score)
    meteor_score = meteor.compute(predictions=pred, references=ref)
    bertscore_score = bertscore.compute(predictions=pred, references=ref, lang="en")

    #  dict
    return {"blue":bleu_score["bleu"],
            "precisions_1":bleu_score["precisions"][0],
            "precisions_2":bleu_score["precisions"][1],
            "precisions_3":bleu_score["precisions"][2],
            "precisions_4":bleu_score["precisions"][3],
            "bp":bleu_score["brevity_penalty"],
            "meteor": meteor_score["meteor"], 
            "bertscore_f1": np.average(bertscore_score["f1"]), 
            "bertscore_p": np.average(bertscore_score["precision"]), 
            "bertscore_r": np.average(bertscore_score["recall"])}


In [40]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./T5",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=12,
    predict_with_generate=True,
    save_strategy="epoch",
    fp16=True,
#     report_to="wandb",
#     logging_dir='./lolololol'

    # push_to_hub=True
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Blue,Precisions 1,Precisions 2,Precisions 3,Precisions 4,Bp,Meteor,Bertscore F1,Bertscore P,Bertscore R
1,No log,0.381750,0.106438,0.529323,0.437168,0.397089,0.373206,0.247340,0.402683,0.905541,0.915388,0.897024
2,No log,0.381557,0.120038,0.499307,0.410628,0.370577,0.346723,0.297953,0.403098,0.904696,0.912898,0.897793
3,No log,0.382017,0.115957,0.495105,0.403252,0.363465,0.340471,0.292477,0.397076,0.904339,0.912713,0.897220
4,No log,0.384120,0.111891,0.513043,0.420339,0.381423,0.359729,0.269781,0.400584,0.904678,0.913017,0.897603
5,No log,0.384805,0.103127,0.498538,0.397260,0.354000,0.330275,0.264369,0.397184,0.904211,0.912280,0.897379
6,No log,0.385710,0.102968,0.499268,0.397942,0.354709,0.331034,0.263469,0.397188,0.904517,0.912729,0.897547
7,No log,0.387828,0.100253,0.498521,0.397569,0.353659,0.329439,0.257178,0.389842,0.904151,0.912661,0.896893
8,No log,0.389030,0.105056,0.481481,0.382060,0.339768,0.314159,0.280648,0.386728,0.902021,0.909714,0.895530
9,No log,0.390439,0.106096,0.476728,0.377668,0.335238,0.309368,0.287010,0.385926,0.901672,0.909097,0.895443
10,No log,0.391527,0.105056,0.481481,0.382060,0.339768,0.314159,0.280648,0.385819,0.902146,0.909979,0.895510


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.10643768175587211, 'precisions': [0.5293233082706766, 0.43716814159292033, 0.3970893970893971, 0.37320574162679426], 'brevity_penalty': 0.24733972532047963, 'length_ratio': 0.41718946047678795, 'translation_length': 665, 'reference_length': 1594}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.10609626295267459, 'precisions': [0.4767277856135402, 0.37766830870279144, 0.3352380952380952, 0.3093681917211329], 'brevity_penalty': 0.2870103636902272, 'length_ratio': 0.44479297365119197, 'translation_length': 709, 'reference_length': 1594}


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.10609626295267459, 'precisions': [0.4767277856135402, 0.37766830870279144, 0.3352380952380952, 0.3093681917211329], 'brevity_penalty': 0.2870103636902272, 'length_ratio': 0.44479297365119197, 'translation_length': 709, 'reference_length': 1594}


TrainOutput(global_step=156, training_loss=0.3604861283913637, metrics={'train_runtime': 316.3504, 'train_samples_per_second': 3.793, 'train_steps_per_second': 0.493, 'total_flos': 730749468672000.0, 'train_loss': 0.3604861283913637, 'epoch': 12.0})

In [41]:
trainer.save_model("T5_overall")

In [42]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [43]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.10952088394040538, 'precisions': [0.4387254901960784, 0.32821229050279327, 0.2845786963434022, 0.2572463768115942], 'brevity_penalty': 0.34180003299598605, 'length_ratio': 0.48226950354609927, 'translation_length': 816, 'reference_length': 1692}
{'eval_loss': 0.3629412055015564, 'eval_blue': 0.10952088394040538, 'eval_precisions_1': 0.4387254901960784, 'eval_precisions_2': 0.32821229050279327, 'eval_precisions_3': 0.2845786963434022, 'eval_precisions_4': 0.2572463768115942, 'eval_bp': 0.34180003299598605, 'eval_meteor': 0.3564716308562745, 'eval_bertscore_f1': 0.890163699388504, 'eval_bertscore_p': 0.8973866164684295, 'eval_bertscore_r': 0.8840654218196868, 'eval_runtime': 11.5238, 'eval_samples_per_second': 8.678, 'eval_steps_per_second': 1.128, 'epoch': 12.0}


In [44]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]

In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_train)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_val)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

In [44]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

[0.14995638702519976,
 0.12970178535938756,
 0.1191005127286421,
 0.10952088394040536]

In [45]:
q='question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?'
c='context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we can throw a little bit.’ " Welker was one of Brady’s favorite targets for six seasons from 2007 to 2012. It’s understandable him and Brady want to meet with both being in the same area. But Brady typically is all business during football season. Welker probably should have known what he was getting into when reaching out to his buddy. "That’s the only thing we really have planned," Welker said of his upcoming workout with Brady. "It’s just funny. I’m sitting there trying to have dinner. ‘Hey, get your ass up here and let’s go throw.’ I’m like, ‘Aw jeez, man.’ He’s going to have me running like 2-minute drills in his backyard or something." Maybe Brady will put a good word in for Welker down in Foxboro if the former Patriots wide receiver impresses him enough.'
enc=tokenizer(q+" "+c,return_tensors='pt',padding=True,truncation=True,max_length=512)
out=model.generate(input_ids=enc['input_ids'].to(device))

In [46]:
tokenizer.decode(out[0]),'how about that morning we go throw?'

('<pad> how about that morning we go throw?</s>',
 'how about that morning we go throw?')

In [47]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/wandb/debug.log
/kaggle/working/wandb/debug-internal.log
/kaggle/working/wandb/run-20240423_202130-gzi1x7vz/run-gzi1x7vz.wandb
/kaggle/working/wandb/run-20240423_202130-gzi1x7vz/files/requirements.txt
/kaggle/working/wandb/run-20240423_202130-gzi1x7vz/files/wandb-metadata.json
/kaggle/working/wandb/run-20240423_202130-gzi1x7vz/files/conda-environment.yaml
/kaggle/working/wandb/run-20240423_202130-gzi1x7vz/files/output.log
/kaggle/working/wandb/run-20240423_202130-gzi1x7vz/files/wandb-summary.json
/kaggle/working/wandb/run-20240423_202130-gzi1x7vz/files/config.yaml
/kaggle/working/wandb/run-20240423_202130-gzi1x7vz/logs/debug.log
/kaggle/working/wandb/run-20240423_202130-gzi1x7vz/logs/debug-internal.log
/kaggle/working/T5/checkpoint-143/trainer_state.json
/kaggle/working/T5/checkpoint-143/spiece.model
/kaggle/working/T5/checkpoint-143/optimizer.pt
/kaggle/working/T5/checkpoint-143/scheduler.pt
/kaggle/working/T5/checkpoint-143/rng_state.pth
/kaggle/working/T5/checkpoint-